<p> <center> <a href="../start_here.ipynb">Home Page</a> </center> </p>

 
<div>
    <span style="float: left; width: 51%; text-align: right;">
        <a href="01_introduction.ipynb">1</a>
        <a>2</a>
        <a href="03_data_transfer.ipynb">3</a>
        <a href="04_tensor_core_util.ipynb">4</a>
        <a href="05_summary.ipynb">5</a>
    </span>
    <span style="float: right; width: 49%; text-align: right;"><a href="03_data_transfer.ipynb">Next Notebook</a></span>
</div>

# Part 1: Optimizing PyTorch's MNIST training program
---

This notebook is focused on optimizing a Deep Neural Network (DNN) training program using MNIST dataset. 


## Running the application

The MNIST database consists of normalized fixed-size handwritten digits images. The database includes 60k training examples and 10k test examples. Click [here](http://yann.lecun.com/exdb/mnist/) to learn more about `LeCun et al., 1998` dataset. In this lab, the MNIST database will be used for training a Deep Neural Network (DNN) that recognizes handwritten digits. Our training program is adopted from [PyTorch GitHub](https://github.com/pytorch/examples/tree/master/mnist) and written using PyTorch framework. The image below is an examples of normalized digits from the testing set.

<center><img src="images/mnist.jpg"></center>
<center><a href="http://yann.lecun.com/exdb/publis/pdf/lecun-90c.pdf"> view source here<a/> </center>
    
Run the cell below to execute the baseline training program.

In [ ]:
!cd ../source_code && time python3 main_baseline.py

**Expected output on DGX A100**:

```python

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308954

Test set: Average loss: 0.1049, Accuracy: 9668/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.166372

Test set: Average loss: 0.0629, Accuracy: 9805/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.116849

Test set: Average loss: 0.0555, Accuracy: 9826/10000 (98%)

---------------------------------------------------------

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.057983

Test set: Average loss: 0.0373, Accuracy: 9862/10000 (99%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.055643

Test set: Average loss: 0.0368, Accuracy: 9869/10000 (99%)


real	1m53.085s
user	2m20.216s
sys	 0m6.212s

```
It takes `~2minutes` to execute the `10 epochs` in the training program

## Train function summary 

<img src="images/train.png" width=60%>

The `train` function is called by the `main` at every epoch. As shown in the screenshot above, three steps of the training process are annotated. At each batch within the `train` function:

- data is copied from CPU to the GPU (device),
- forward pass runs on the GPU,
- backward pass runs on the GPU.

These steps are repeated for 10 epochs of the training program here: [main_baseline.py](../source_code/main_baseline.py). In the rest of the notebook, the expression `training program` will be substituted for the word `application` interchangeably.   


## Profiling application

The first step to use the NVIDIA Nsight Systems command line interface (CLI) to profile our application. The rule of thumb is to limit the profiling to let's say the second or third epoch. Knowing fully well that the first epoch is meant to warm up the GPU and to get things going, second and subsequent epochs would be of the same workload. We will be using the APIs `cudaProfilerStart` to set profiling start point and `cudaProfilerStop` to mark stop region within our application as shown below. In this section of the lab, our profiling will be limited to just the `second epoch` throughout the optimization workload.

<img src=images/cudaprofilestart.png width=60%>

Please run the cell below to profile the application.

In [ ]:
!nsys profile --trace cuda,osrt \
--capture-range cudaProfilerApi \
--output ../reports/baseline \
--force-overwrite true \
python3 ../source_code/main_baseline.py

Let's have a brief on the profiling command line used.

- `nsys`: command to call Nsight Systems CLI 
- `profile`: command to instruct profiling
- `--trace`: flag that represent tracing of APIs 
- `cuda,osrt`: APIs to be traced 
- `--capture-range`:flag that indicate start and stop profiling range using `cudaProfilerStart()` and `CudaProfilerStop()` APIs. 
- `--output`: flag that indicate the output of the profile  
- `../reports/baseline`: directory to save the profile output as `basline.qdrep` or `baseline.nsys-rep` 

- `--force-overwrite`: flag that denotes overwrite existing output file. It is usually set as `true` or `false`

- `python3`: command to run the DNN code in linux/ubuntu OS
- `../source_code/main_baseline.py`: path to the DNN python code

A detailed user guide on Nsight Systems CLI profiling commands is presented at the summary notebook of this material. 


Once the profiling is done we will inspect the report in the Nsight Systems' GUI. Download the [baseline.nsys-rep](../reports/baseline.nsys-rep) file at `../reports/baseline.nsys-rep` and view in the NVIDIA Nsight Systems GUI. 

## Analyze the profile report 

**Application timeline** 

<img src=images/report_baseline.jpg>

Hovering over the blue chart in the CUDA device row, we see that the CUDA kernel coverage on the GPU is about `35-40%` throughout. This means that the GPU is idle for the remaining `65-70%` of the time.Also, GPU idle intervals is can be seen within the kernel coverage. 

<img src=images/report_kernel_coverage.jpg>

Let's zoom in to the timeline by highlighting the region we want to zoom into and then press Shift+Z keys.

<img src=images/report_gpu_starvation.jpg>

Clearly there is a repeating pattern where the GPU is idle or starved for some time followed by a burst of kernel and memory operations. During the GPU starvation period, some of the CPU threads are performing `read/write` operation or waiting on a `condition variable`. The is GPU starvation is due to sluggish CPU threads. To identify this, we will use [NVIDIA Tools Extension](https://nvidia.github.io/NVTX/doxygen/index.html) (NVTX) library which provides a set of functions for annotating events, code ranges, and resources in your application. The NVTX APIs provide additional information to NVIDIA’s tools while incurring almost no overhead when the tool is not attached to the application. Sample code to add NVTX annotations looks like the following:


```python
    from torch.cuda import nvtx
    
    ....

    nvtx.range_push("annotation string")

     #lines of code to annotate

    nvtx.range_pop(); 
    
    .....

```

<img src=images/nvtx_annotation.jpg width=60%>

To see NVTX annotations added, run the cell below. You will notice the changes written in `green color` in the [main_baseline_nvtx.py](../source_code/main_baseline_nvtx.py) code on the right column.

In [1]:
!diff -y -d --color=always ../source_code/main_baseline.py ../source_code/main_baseline_nvtx.py

'diff' is not recognized as an internal or external command,
operable program or batch file.


Run the cell below to profile again and see the effect of the newly added NVTX annotations.

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--output ../reports/baseline_nvtx \
--force-overwrite true \
python3 ../source_code/main_baseline_nvtx.py

The inclusion of `nvtx` in the `--trace` flag enables the trace of the code where `nvtx` annotation is specified. Open the report with NVTX annotations `(baseline_nvtx.nsys-rep file)` in the GUI and zoom in to find the repeating pattern of 'GPU idle' followed by `a burst of kernel and memory operations`. From the NVTX annotation in the report shown below, we can see that the GPU is idle during the `Data loading` phase of the application. Also, the time taken for `epoch 2 (28.035s)` and `training (23.963s)` is given. 

<img src=images/report_nvtx.png>

## Optimize Code to Address the CPU Bottleneck
From the profile report it is obvious that the data loading step of the application is the cause of the repeated GPU idle/starvation pattern. Let's inspect the data loader [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) used in our application. From the code in `main_baseline_nvtx.py` shown below, a single worker subprocess is asynchronously loading the data.


<img src=images/NumberOfWorkers.jpg width=50%>

In order to increase the overlap between data loading and training on the GPU, the `num_workers` parameter should be increased. Run the following cell to see the code changes made to tune this parameter

In [ ]:
!diff -d --color=always ../source_code/main_baseline_nvtx.py ../source_code/main_opt1.py

Depending on the number of CPU cores available on the target system, we can increase `num_of_workers` to  use the total number of CPU cores available by setting the `num_workers` to `multiprocessing.cpu_count()`, `2`, and `more`, to improve the overlap. There are rare situations where setting `num_workers` to `multiprocessing.cpu_count()` would prompt the following warnings:

```python
/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 256 worker processes in total. 
Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. 
Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
```

<br/>
You have to adjust to the recommended max number of workers. Another is seeing warning like CPU tracing is unsupported as shown below

```python
GPU 0: General Metrics for NVIDIA GA100 (any frequency)
WARNING: CPU context switch tracing not supported, disabling.
Try the 'nsys status --environment' command to learn more.

WARNING: CPU sampling not supported, disabling.
Try the 'nsys status --environment' command to learn more.
```

Despite the warnings, you can however proceed to profile your code successfully.  

## Profile to Verify Optimization
The next step is to profile again and verify if the code change addresses the bottleneck. Please, run the cell below.

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--output ../reports/firstOptimization \
--force-overwrite true \
python3 ../source_code/main_opt1.py

Open the report `firstOptimization.nsys-rep` in the GUI and zoom into the data loading phase of one of the batches. Comparing the profiles from before and after the optimization, we notice that the `data loading` phase's duration, `epoch`, and `train` time have reduced. This might not seem like a lot, but it adds up when you consider hundreds of batches over each epoch in the training program.

<img src=images/report_firstoptimization.jpg>

Now that we have addressed one bottleneck, let's identify the next one `(Data Transfers between Host and Device)` in the next notebook.

## Links and Resources


[NVIDIA® Nsight™ Systems](https://docs.nvidia.com/nsight-systems/)


**NOTE**: To be able to see the profiler output, please download NVIDIA Nsight Systems' latest version from [here](https://developer.nvidia.com/nsight-systems).

You can also get resources from [openhackathons technical resource page](https://www.openhackathons.org/s/technical-resources)

---
## Licensing
  
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

<div>
    <span style="float: left; width: 51%; text-align: right;">
        <a href="01_introduction.ipynb">1</a>
        <a >2</a>
        <a href="03_data_transfer.ipynb">3</a>
        <a href="04_tensor_core_util.ipynb">4</a>
        <a href="05_summary.ipynb">5</a>
    </span>
    <span style="float: left; width: 49%; text-align: right;"><a href="03_data_transfer.ipynb">Next Notebook</a></span>
</div>

<br>
<p> <center> <a href="../start_here.ipynb">Home Page</a> </center> </p>